<a href="https://colab.research.google.com/github/mayurc96/personal_project/blob/main/Ecommerce_Charges_varification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**
Please understand the below mentioned real-life scenario and try to solve the assignment.
The sample data is attached in the link provided below for your reference.


## **Business Scenario**
You are a data analyst and your client has a large ecommerce company in India (let’s call it X).
X gets a thousand orders via their website on a daily basis and they have to deliver them as fast
as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India as delivery partners who charge them some amount per delivery.
The charges are dependent upon two factors:
● Weight of the product and the Price of the product.
● Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their Delivery partners per Order are correct.

# **Solution**

# Import all the neccesary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mplot
import math

# Import all the excel files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
order_report_df = pd.read_excel('/content/drive/MyDrive/Cointab Assignment details/Company X - Order Report.xlsx')
sku_master_df = pd.read_excel('/content/drive/MyDrive/Cointab Assignment details/Company X - SKU Master.xlsx')
courier_company_invoice_df = pd.read_excel('/content/drive/MyDrive/Cointab Assignment details/Courier Company - Invoice.xlsx')
courier_company_rates_df = pd.read_excel('/content/drive/MyDrive/Cointab Assignment details/Courier Company - Rates.xlsx')
pincode_zones_df = pd.read_excel('/content/drive/MyDrive/Cointab Assignment details/Company X - Pincode Zones.xlsx')

# Understand the data

In [ ]:
# First look of Order Report
order_report_df

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286
396,2001806229,8904223818850,1,COD,563
397,2001806226,8904223818850,2,COD,1125
398,2001806210,8904223816214,1,COD,402


In [ ]:
# first look of SKU Master
sku_master_df

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [ ]:
# First look of courier_company_invoice_df
courier_company_invoice_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


In [ ]:
# First look of courier_company_rates_df
courier_company_rates_df

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.25,29.5,23.6,13.6,23.6
1,B,0.50,33.0,28.3,20.5,28.3
2,C,0.75,40.1,38.9,31.9,38.9
3,D,1.25,45.4,44.8,41.3,44.8
4,E,1.50,56.6,55.5,50.7,55.5


In [ ]:
# First look of pincode_zones_df
pincode_zones_df

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


# Data Preparation

In [ ]:
# Renaming column name in order_report_df
order_report_df = order_report_df.rename(columns={'ExternOrderNo': 'Order ID'})

In [ ]:
# Renaming column name in courier_company_invoice_df
courier_company_invoice_df = courier_company_invoice_df.rename(columns={'Charged Weight': 'Total weight as per Courier Company (KG)','Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'})

In [ ]:
# Change the zone column in to lower case
courier_company_rates_df['Zone'] = courier_company_rates_df['Zone'].str.lower()
courier_company_rates_df

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.25,29.5,23.6,13.6,23.6
1,b,0.50,33.0,28.3,20.5,28.3
2,c,0.75,40.1,38.9,31.9,38.9
3,d,1.25,45.4,44.8,41.3,44.8
4,e,1.50,56.6,55.5,50.7,55.5


# Data Wrangling

## Find out total weight & total order amount
To find the weight of every product merge Order Report and SKU Master. In order to find total order amount and total weight for each order group by order id and perform aggregate operration or item price and weight. Additionly convert the weight into kg.

In [ ]:
# Merge  Order Report & SKU Master
order_sku_df = pd.merge(order_report_df, sku_master_df, on='SKU', how= 'inner')

In [ ]:
# Drop the duplicate rows
order_sku_df=order_sku_df.drop_duplicates()

In [ ]:
# Find out the total weight by multiplying 'Order Qrt' and 'Weight (g)' columns
order_sku_df_copy = order_sku_df.copy()
order_sku_df_copy.loc[:, 'total_weight'] = order_sku_df['Order Qty'] * order_sku_df['Weight (g)']

In [ ]:
# Use Group by to  perform aggregate  operation to find total weight and total amount per order
grouped_order_report_df = order_sku_df_copy.groupby(['Order ID','Payment Mode']).agg({'Item Price(Per Qty.)':'sum', 'total_weight':'sum'}).reset_index()
grouped_order_report_df= grouped_order_report_df.sort_values(by='Order ID')

In [ ]:
# Convert the total_weight in kg
grouped_order_report_df['total_weight'] =np.round((grouped_order_report_df['total_weight']/1000),2)

In [ ]:
# Rename the column 'Item Price(Per Qty.)' to 'order_amount' as we got total amount per order
grouped_order_report_df = grouped_order_report_df.rename(columns={'Item Price(Per Qty.)':'order_amount'})

In [ ]:
grouped_order_report_df

,Order ID,Payment Mode,order_amount,total_weight
0,2001806210,COD,1324,0.22
1,2001806226,COD,1125,0.48
2,2001806229,COD,1082,0.50
3,2001806232,COD,2139,1.30
4,2001806233,Prepaid,1486,0.24
...,...,...,...,...
119,2001821995,Prepaid,484,0.48
120,2001822466,COD,9019,1.38
121,2001823564,Prepaid,4803,0.67
122,2001825261,Prepaid,9049,1.56


## Find out zones as per x
In order to find out zones of every order, merge 'Customer Pincode' and 'Type of Shipment' columns of courier_company_invoice_df with grouped_order_report_df on 'order id' column. Then merge pincode_zones_df to get zone for every order

In [ ]:
# Merge the nessecary columns of courier_company_invoice_df with grouped_order_report_df
x_order_report_df = pd.merge(grouped_order_report_df,courier_company_invoice_df[['Order ID','Customer Pincode','Type of Shipment']],on='Order ID', how='left')


In [ ]:
# Merge the zone column of pincode_zones_df with x_order_report_df
x_order_report_df = pd.merge(x_order_report_df,pincode_zones_df[['Customer Pincode','Zone']],on='Customer Pincode', how='inner')

In [ ]:
# Drop  the unnecessary columns
x_order_report_df.drop(columns=['Customer Pincode'], inplace=True)

In [ ]:
x_order_report_df

,Order ID,Payment Mode,order_amount,total_weight,Type of Shipment,Zone
0,2001806210,COD,1324,0.22,Forward charges,b
1,2001806226,COD,1125,0.48,Forward charges,d
2,2001806229,COD,1082,0.50,Forward charges,d
3,2001806232,COD,2139,1.30,Forward charges,d
4,2001806233,Prepaid,1486,0.24,Forward charges,b
...,...,...,...,...,...,...
169,2001821995,Prepaid,484,0.48,Forward charges,b
170,2001822466,COD,9019,1.38,Forward charges,b
171,2001823564,Prepaid,4803,0.67,Forward and RTO charges,d
172,2001825261,Prepaid,9049,1.56,Forward and RTO charges,d


## Calculate  the COD charge per order
For order amount equal to or less than 300 COD will 15.
For order amount greater than 300 COD is 5% of order amount.

In [ ]:
# Define a custom function to calculate the cod_charge
def calculate_cod_charge(row):
    if row['Payment Mode'] == 'COD':
        if row['order_amount'] <= 300:
            return 15
        else:
            return row['order_amount'] * 0.05
    else:
        return 0  # No cod_charge if Payment Mode is not 'cod'

In [ ]:
# Apply the calculate_cod_charge function to get cod charge for every order
x_order_report_df['cod_charge'] = x_order_report_df.apply(calculate_cod_charge, axis=1)

In [ ]:
x_order_report_df

,Order ID,Payment Mode,order_amount,total_weight,Type of Shipment,Zone,cod_charge
0,2001806210,COD,1324,0.22,Forward charges,b,66.20
1,2001806226,COD,1125,0.48,Forward charges,d,56.25
2,2001806229,COD,1082,0.50,Forward charges,d,54.10
3,2001806232,COD,2139,1.30,Forward charges,d,106.95
4,2001806233,Prepaid,1486,0.24,Forward charges,b,0.00
...,...,...,...,...,...,...,...
169,2001821995,Prepaid,484,0.48,Forward charges,b,0.00
170,2001822466,COD,9019,1.38,Forward charges,b,450.95
171,2001823564,Prepaid,4803,0.67,Forward and RTO charges,d,0.00
172,2001825261,Prepaid,9049,1.56,Forward and RTO charges,d,0.00


## Calculate the Weight slab as per X (KG)
To calculate Weight slab as per X, merge courier_company_rates_df to get rates for diff zones

In [ ]:
# Merge the courier_company_rates_df with x_order_report_df to get  rates
x_order_report_df = pd.merge(x_order_report_df,courier_company_rates_df,on='Zone', how='left')


In [ ]:
# Drop the duplicates
x_order_report_df = x_order_report_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Calculate the chargable weight slab as per x (For how much weight x has to pay to courier company)
x_order_report_df['charged_weight_as_per_x'] = np.ceil(x_order_report_df['total_weight'] / x_order_report_df['Weight Slabs']) * x_order_report_df['Weight Slabs']


In [ ]:
x_order_report_df

,Order ID,Payment Mode,order_amount,total_weight,Type of Shipment,Zone,cod_charge,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,charged_weight_as_per_x
0,2001806210,COD,1324,0.22,Forward charges,b,66.20,0.50,33.0,28.3,20.5,28.3,0.50
1,2001806226,COD,1125,0.48,Forward charges,d,56.25,1.25,45.4,44.8,41.3,44.8,1.25
2,2001806229,COD,1082,0.50,Forward charges,d,54.10,1.25,45.4,44.8,41.3,44.8,1.25
3,2001806232,COD,2139,1.30,Forward charges,d,106.95,1.25,45.4,44.8,41.3,44.8,2.50
4,2001806233,Prepaid,1486,0.24,Forward charges,b,0.00,0.50,33.0,28.3,20.5,28.3,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,Prepaid,484,0.48,Forward charges,b,0.00,0.50,33.0,28.3,20.5,28.3,0.50
120,2001822466,COD,9019,1.38,Forward charges,b,450.95,0.50,33.0,28.3,20.5,28.3,1.50
121,2001823564,Prepaid,4803,0.67,Forward and RTO charges,d,0.00,1.25,45.4,44.8,41.3,44.8,1.25
122,2001825261,Prepaid,9049,1.56,Forward and RTO charges,d,0.00,1.25,45.4,44.8,41.3,44.8,2.50


## Calculate the charges X has to pay as per weight slabs


In [ ]:
# Define a function to calculate charges x has to pay as for weight as per weight slabs per zone
def calculate_weight_charges(row):
  if row['Type of Shipment']== 'Forward and RTO charges':
    if (row['charged_weight_as_per_x']/row['Weight Slabs']) > 1:
      return (row['Forward Fixed Charge']+ row['RTO Fixed Charge'] + (row['Forward Additional Weight Slab Charge']*((row['charged_weight_as_per_x']/row['Weight Slabs'])-1))+(row['RTO Additional Weight Slab Charge']*((row['charged_weight_as_per_x']/row['Weight Slabs'])-1)))
    else:
      return (row['Forward Fixed Charge']+ row['RTO Fixed Charge'])

  else:
    if (row['charged_weight_as_per_x']/row['Weight Slabs']) > 1:
      return (row['Forward Fixed Charge']+ (row['Forward Additional Weight Slab Charge']*((row['charged_weight_as_per_x']/row['Weight Slabs'])-1)))
    else:
      return row['Forward Fixed Charge']

In [ ]:
# Apply calculate_weight_charges function to calculate charges per order
x_order_report_df['weight_charges'] = x_order_report_df.apply(calculate_weight_charges, axis=1)

In [ ]:
x_order_report_df

,Order ID,Payment Mode,order_amount,total_weight,Type of Shipment,Zone,cod_charge,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,charged_weight_as_per_x,weight_charges
0,2001806210,COD,1324,0.22,Forward charges,b,66.20,0.50,33.0,28.3,20.5,28.3,0.50,33.0
1,2001806226,COD,1125,0.48,Forward charges,d,56.25,1.25,45.4,44.8,41.3,44.8,1.25,45.4
2,2001806229,COD,1082,0.50,Forward charges,d,54.10,1.25,45.4,44.8,41.3,44.8,1.25,45.4
3,2001806232,COD,2139,1.30,Forward charges,d,106.95,1.25,45.4,44.8,41.3,44.8,2.50,90.2
4,2001806233,Prepaid,1486,0.24,Forward charges,b,0.00,0.50,33.0,28.3,20.5,28.3,0.50,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,Prepaid,484,0.48,Forward charges,b,0.00,0.50,33.0,28.3,20.5,28.3,0.50,33.0
120,2001822466,COD,9019,1.38,Forward charges,b,450.95,0.50,33.0,28.3,20.5,28.3,1.50,89.6
121,2001823564,Prepaid,4803,0.67,Forward and RTO charges,d,0.00,1.25,45.4,44.8,41.3,44.8,1.25,86.7
122,2001825261,Prepaid,9049,1.56,Forward and RTO charges,d,0.00,1.25,45.4,44.8,41.3,44.8,2.50,176.3


## Calculate Total Expected Charge as per X (Rs.)

In [ ]:
# Calculate the 'Expected Charge as per X (Rs.)'
x_order_report_df['Expected Charge as per X (Rs.)'] = x_order_report_df['cod_charge'] + x_order_report_df['weight_charges']

# Final Output
Drop unnecessary columns since calculations have been completed. To compare the final output with the courier company's data, we will merge the required columns from the courier_company_invoice_df. Additionally, rename the columns from x_order_report_df according to the output requirements.






In [ ]:
# Drop the Unnecessary columns
x_order_report_df.drop(columns=['Payment Mode', 'Type of Shipment','Forward Fixed Charge','Forward Additional Weight Slab Charge','RTO Fixed Charge','RTO Additional Weight Slab Charge','cod_charge','Weight Slabs','weight_charges','order_amount'], inplace=True)

In [ ]:
# Reaname coulmns
x_order_report_df.rename(columns={'total_weight':'Total weight as per X (KG)','Zone':'Delivery Zone as per X', 'charged_weight_as_per_x' : 'Weight slab as per X (KG)'},inplace=True)


AS we dont have 'Weight slab charged by Courier Company (KG)', we have to merge 'Weight Slabs' column from courier_company_rates_df with on zone.

In [ ]:
# Merge courier_company_invoice_df & courier_company_rates_df
courier_company_invoice_df = pd.merge(courier_company_invoice_df, courier_company_rates_df[['Zone','Weight Slabs']], on ='Zone', how = 'left')

In [ ]:
# Find out 'Weight slab charged by Courier Company (KG)'
courier_company_invoice_df['Weight slab charged by Courier Company (KG)'] = np.ceil(courier_company_invoice_df['Total weight as per Courier Company (KG)'] / courier_company_invoice_df['Weight Slabs']) * courier_company_invoice_df['Weight Slabs']


In [ ]:
# Renaming column name in courier_company_invoice_df
courier_company_invoice_df = courier_company_invoice_df.rename(columns={'Zone':'Delivery Zone charged by Courier Company'})


In [ ]:
# Merge the courier_company_invoice_df and x_order_report_df
output_data = pd.merge(x_order_report_df,courier_company_invoice_df[['AWB Code','Order ID','Total weight as per Courier Company (KG)','Delivery Zone charged by Courier Company','Weight slab charged by Courier Company (KG)','Charges Billed by Courier Company (Rs.)']],on='Order ID', how='left')

In [ ]:
# Rearrange the columns as per output requirement
output_data = output_data[['Order ID','AWB Code','Total weight as per X (KG)','Weight slab as per X (KG)','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','Delivery Zone as per X','Delivery Zone charged by Courier Company','Expected Charge as per X (Rs.)','Charges Billed by Courier Company (Rs.)']]

In [ ]:
# find out the Difference Between Expected Charges and Billed Charges (Rs.)
final_output_data= output_data.copy()
final_output_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = output_data['Expected Charge as per X (Rs.)'] - output_data['Charges Billed by Courier Company (Rs.)'].values


In [ ]:
final_output_data

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.50,2.92,3.00,b,b,99.20,174.5,-75.30
1,2001806226,1091117222065,0.48,1.25,0.68,1.25,d,d,101.65,90.2,11.45
2,2001806229,1091117222080,0.50,1.25,0.71,1.25,d,d,99.50,90.2,9.30
3,2001806232,1091117222124,1.30,2.50,1.30,2.50,d,d,197.15,140.0,57.15
4,2001806233,1091117222135,0.24,0.50,0.78,1.00,b,b,33.00,61.3,-28.30
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.48,0.50,0.50,1.25,b,d,33.00,45.4,-12.40
120,2001822466,1091121305541,1.38,1.50,1.10,1.25,b,d,540.55,135.0,405.55
121,2001823564,1091121666133,0.67,1.25,0.70,1.25,d,d,86.70,172.8,-86.10
122,2001825261,1091121981575,1.56,2.50,1.60,2.50,d,d,176.30,345.0,-168.70


In [ ]:
# Save the DataFrame to an Excel file
final_output_data.to_excel('final_output_data.xlsx', index=False)